In [1]:
import os
import requests
import time
import pandas as pd
from io import BytesIO
from datetime import datetime

In [2]:
URL_FIPEZAP = "https://downloads.fipe.org.br/indices/fipezap/fipezap-serieshistoricas.xlsx"
FULL_PATH = os.getcwd()
DT_BASE = datetime.now().strftime("%Y-%m-%d")

In [3]:
class FipeZap:
    
    def __init__(self, url):
        self.url = url
            
    def get_data_excel(self):
        
        max_tentativas = 5
        intervalo = 15
        
        for tentativa in range(max_tentativas):
            
            response = requests.get(self.url, verify=False)
            
            if response.status_code == 200:
                
                content = response.content
                
                df = pd.read_excel(BytesIO(content), sheet_name="Índice FipeZAP", header=3, usecols=[1, 2, 7], engine="openpyxl")
                
                ultimo_indice = df['Total'].last_valid_index()
                df = df.iloc[:ultimo_indice + 1]
                df['DT_REG_IC'] = pd.to_datetime(df['Data'])
                df['NM_IC'] = "FIPEZAP"
                df['VL_IC'] = df['Total']
                df['VL_VRC_IC'] = df['Total.1'] * 100
                df['DT_REF'] = f"{DT_BASE}"
                df = df.drop(['Data', 'Total', 'Total.1'], axis = 1)
                    
                print(f"Requisição bem-sucedida! Status code: {response.status_code}")
                
                return df
            
            else:
                
                print(f"Tentativa {tentativa + 1}: Status code {response.status_code}. Tentando novamente em {intervalo} segundos...")
                time.sleep(intervalo)
                
        print(f"Não foi possível obter uma resposta 200 após {max_tentativas} tentativas.")
        
        return None
        

In [4]:
fipezap = FipeZap(f"{URL_FIPEZAP}")
df = fipezap.get_data_excel()

if~(df is None):
    df.to_csv(f"{FULL_PATH}/dados_fipezap.csv", index = False)
else:
    print(f"Não foi possível alcançar a url: {URL_FIPEZAP}.")

/root/projects/hands-on-notebooks/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'downloads.fipe.org.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Requisição bem-sucedida! Status code: 200
